In [118]:
import requests as rq
from bs4 import BeautifulSoup
import json

### Task

Collect a tabular dataset of all battles of WWII from wikipedia.
for each battle collect it's theater, location, date, outcomes and as much of additional info as there is

In [119]:
base_url = 'https://en.wikipedia.org/wiki/List_of_World_War_II_battles'

In [121]:
def _default_collect(url):
    '''returns document from url as a BS DOM'''
    response = rq.get(url)
    response.raise_for_status()
    return BeautifulSoup(response.content, 'html.parser')

In [122]:
soup = _default_collect(base_url)

In [123]:
content = soup.find('div', id='mw-content-text')

In [124]:
fronts = [el.text [:-6]for el in content.find_all('h2')[1:-1]]

In [125]:
len(content.find_all("div", "div-col columns column-width"))

9

In [126]:
ul = content.find_all("div", "div-col columns column-width")[0].ul

In [138]:
def dictify(ul, level=0):
    result = dict()
    
    for li in ul.find_all("li", recursive=False):
        text = li.stripped_strings
        key = next(text)
        try:
            time = next(text)
        except StopIteration:
            time = None

        ul, link = li.find("ul"), li.find('a')
        if link:
            link = 'https://en.wikipedia.org' + link.get('href')
            
        nextlevel = level + 1

        r ={'url': link,
            'time':time,
            'level': nextlevel} 
        
        if ul:
            r['children'] = dictify(ul, level=nextlevel+1)

        result[key] = r
    return result

In [139]:
# data = dictify(ul)
# list(ul.li.li.stripped_strings)

In [140]:
theaters = {}
for front, div in zip(fronts, content.find_all("div", "div-col columns column-width")):
    theaters[front] = dictify(div.ul)

In [141]:
with open('all_battles.json', 'w') as f:
    json.dump(theaters, f)